In [1]:
# importing data and auxiliar functions

import saopaulo.sp_grid as gr
import bikescience.distributions as dist
import saopaulo.cycling_potential as cp
import saopaulo.choropleth_folium as choro_folium
import saopaulo.choropleth as choro
from bikescience.slope import plot_slope, plot_slopes, split_route
from shapely.geometry import LineString
import ast
from numpy import nan

import folium
import pandas as pd
import geopandas as gpd
from ipywidgets import interact_manual, widgets, fixed
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import warnings
warnings.simplefilter('ignore')

/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
#read data

zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
zone_shp.crs = {'init': 'epsg:31983'}  
zone_shp.to_crs(epsg='4326', inplace=True)

# read routes files and join with general trips file
od_trips = pd.read_csv('bases/complete_csv.csv')
# fix FE_VIA with wrong format (because of thousand separator)
od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] = od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] / 1000

od_trips.set_index('ID_ORDEM', inplace=True, drop=False)

od_trips.dropna(inplace = True)

the_grid = gr.create(n=9, west_offset=-0.15, east_offset=0.23, north_offset=0.19, south_offset=-0.46)

In [3]:
od_trips = gpd.GeoDataFrame(od_trips, crs={'init': 'epsg:4326'},
                 geometry=[LineString(eval(r)) for r in od_trips['route']])

In [4]:
def calculate_trips_zone_intersection (trips, zones):
    
    trips_geometries = list(trips['geometry'])
    trips_expansion = list(trips['FE_VIA'])

    trips_per_zone = [0]*517 # hash to store trips indexed by zones

    progress = 0
    
    for z in range(len(zones)):
        zone = zones.iloc[z]['geometry']
        
        if z / len(zones) >= progress:
            print (round(progress * 100), '%')
            progress += 0.1
            
        for i in range (len (trips_geometries)):

            if (trips_geometries[i].intersects(zone)):
                trips_per_zone[z] += trips_expansion[i]
            
    return trips_per_zone   

def calculate_trips_mean_pot_intersection (trips, zones, potential_label, pot_as_prob = False):
    
    trips_geometries = list(trips['geometry'])
    trips_expansion = list(trips['FE_VIA'])
    trips_potential = list(trips[potential_label])

    total_per_zone = [0]*517 # hash to store trips indexed by zones

    progress = 0
    
    for z in range(len(zones)):
        total_trips = 0
        zone = zones.iloc[z]['geometry']
        
        if z / len(zones) >= progress:
            print (round(progress * 100), '%')
            progress += 0.1
            
        for i in range (len (trips_geometries)):

            if (trips_geometries[i].intersects(zone)):
                total_per_zone[z] += trips_expansion[i] * trips_potential[i]
                total_trips += trips_expansion[i]
        
        if not pot_as_prob:
            total_per_zone[z] = 0 if total_trips == 0 else total_per_zone[z] / total_trips
            
    return total_per_zone    


def plot_trips_per_zone_intersection (trips, zones):
    aux = calculate_trips_zone_intersection(trips, zone_shp)
    zone_shp['intersect_trips'] = aux


    tooltip_columns = ['NomeZona', 'intersect_trips']
    tooltip_aliases = ['Zone', 'Trips']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, 'Trips per zone', 
                                 'YlOrBr', lambda x : x['intersect_trips'], 
                          zone_shp.loc[zone_shp['NumeroMuni']==36], tooltip_columns, tooltip_aliases)
    

In [5]:
def filter_potential_widgets(btn_text, process_function):
    
    width = '200px'
    layout = widgets.Layout(width = '500px')
    
    im = interact_manual(
        process_function,
        final_potential = widgets.FloatRangeSlider(
            value=[0.85, +1.],
            min=0, max=1., step=0.05,
            description='Final Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Distance Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        inclination_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Inclination Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        age_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Age Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        reference = widgets.Dropdown(options = [('Whole Trip', 'intersect'),('Origin', 'ZONA_O'), 
                                                ('Destination', 'ZONA_D')],
                                     description = 'Trip Reference Point:\t', 
                                     style={'description_width': width}),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Modal', 
                                 style={'description_width': width})
    )
    im.widget.children[8].description = btn_text
    
def inside_limits (limits, value):
    return value >= limits[0] and value <= limits[1]
    
def filter_potential(final_potential, distance_potential, age_potential, inclination_potential, modal):
    if modal != None:
        print(modal)
        aux = od_trips.loc[od_trips['modal'] == modal]
    else:
        aux = od_trips
    
    aux = aux.loc[aux['age_potential'] >= age_potential[0]]
    aux = aux.loc[aux['age_potential'] <= age_potential[1]]
    aux = aux.loc[aux['distance_potential'] >= distance_potential[0]]
    aux = aux.loc[aux['distance_potential'] <= distance_potential[1]]
    aux = aux.loc[aux['inclination_potential'] >= inclination_potential[0]]
    aux = aux.loc[aux['inclination_potential'] <= inclination_potential[1]]
    aux = aux.loc[aux['final_potential'] >= final_potential[0]]
    aux = aux.loc[aux['final_potential'] <= final_potential[1]]
    return aux

In [6]:
def plot_potential_trips_per_zone(final_potential, distance_potential, age_potential, 
                                  inclination_potential, reference, modal, debug=False, only_sp = True):
    df = filter_potential(final_potential, distance_potential, age_potential, inclination_potential, modal)
    
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if only_sp else zone_shp
    
    if (reference == 'intersect'):
        trips_per_zone = calculate_trips_zone_intersection(df, zones)
    else:
        trips_per_zone = [0]*517
        for i, t in df.iterrows():
            trips_per_zone[int(t[reference] - 1)] += t['FE_VIA']

    zone_shp['potential_trips'] = trips_per_zone
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if only_sp else zone_shp

    tooltip_columns = ['NomeZona', 'potential_trips']
    tooltip_aliases = ['Zone', 'Potential Trips']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, 'Trips per zone', 'YlOrBr', lambda x : x['potential_trips'], 
                          zones, tooltip_columns, tooltip_aliases)
    
    if debug:
        #display(df)
        plot_slopes (fmap, df['geometry'], 1000, False)
    
    display(fmap)


In [7]:
filter_potential_widgets('Filter trips', plot_potential_trips_per_zone)

interactive(children=(FloatRangeSlider(value=(0.85, 1.0), description='Final Potential:\t', layout=Layout(widt…

In [8]:
def plot_potential_mean_widgets():
    im = interact_manual(
        plot_potential_mean,
        variable = widgets.Dropdown(options = [('Final', 'final_potential'), 
                                               ('Distance', 'distance_potential'), 
                                               ('Age', 'age_potential'), 
                                               ('Inclination', 'inclination_potential')],
                                    description = 'Potential Variable', 
                                    style={'description_width': '300px'},
                                    layout = widgets.Layout(width = '500px')),
        reference = widgets.Dropdown(options = [('Origin', 'ZONA_O'), ('Destination', 'ZONA_D'), 
                                                ('Whole Trip', 'intersect')],
                                     description = 'Trip Reference Point:\t', 
                                    style={'description_width': '300px'},
                                    layout = widgets.Layout(width = '500px')),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Modal', 
                                    style={'description_width': '300px'},
                                    layout = widgets.Layout(width = '500px'))
    )
    im.widget.children[4].description = 'Plot Map'
    
def plot_potential_mean(variable, reference, modal, only_sp = True):
    trips = [0]*517
    potential = [0]*517
    
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if only_sp else zone_shp

    if modal != None:
        aux = od_trips.loc[od_trips['modal'] == modal]
    else:
        aux = od_trips
        
    if reference == 'intersect':
        mean = calculate_trips_mean_pot_intersection(aux, 
                                                    zones, variable)
    else:
        for i, t in aux.iterrows():
            potential[int(t[reference] - 1)] += t[variable] * t['FE_VIA']
            trips[int(t[reference] - 1)] += t['FE_VIA']

        mean = [0 if trips[i] == 0 else potential[i] / trips [i] for i in range(517)]
    
    zone_shp['mean_potential'] = mean
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if only_sp else zone_shp

    tooltip_columns = ['NomeZona', 'mean_potential']
    tooltip_aliases = ['Zone', 'Mean Potential (' + reference + ')']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, 'Mean Potential (' + reference + ') ' + ' (' + variable + ')', 
                                 'YlOrBr', lambda x : x['mean_potential'], 
                          zones, tooltip_columns, tooltip_aliases)
    display(fmap)

In [9]:
plot_potential_mean_widgets()

interactive(children=(Dropdown(description='Potential Variable', layout=Layout(width='500px'), options=(('Fina…

In [10]:
def plot_potential_prob_widgets():
    im = interact_manual(
        plot_potential_prob,
        variable = widgets.Dropdown(options = [('Final', 'final_potential'), 
                                               ('Distance', 'distance_potential'), 
                                               ('Age', 'age_potential'), 
                                               ('Inclination', 'inclination_potential')],
                                    description = 'Potential Variable', 
                                    style={'description_width': '300px'},
                                    layout = widgets.Layout(width = '500px')),
        reference = widgets.Dropdown(options = [('Origin', 'ZONA_O'), ('Destination', 'ZONA_D'), 
                                                ('Whole Trip', 'intersect')],
                                     description = 'Trip Reference Point:\t', 
                                    style={'description_width': '300px'},
                                    layout = widgets.Layout(width = '500px')),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Modal', 
                                    style={'description_width': '300px'},
                                    layout = widgets.Layout(width = '500px'))
    )
    im.widget.children[4].description = 'Plot Map'
    
def plot_potential_prob(variable, reference, modal, only_sp = True):
    trips = [0]*517
    potential = [0]*517
    
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if only_sp else zone_shp

    if modal != None:
        aux = od_trips.loc[od_trips['modal'] == modal]
    else:
        aux = od_trips
        
    if reference == 'intersect':
        potential = calculate_trips_mean_pot_intersection(aux, 
                                                    zones, variable, True)
    else:
        for i, t in aux.iterrows():
            potential[int(t[reference] - 1)] += t[variable] * t['FE_VIA']
    
    zone_shp['mean_potential'] = potential
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if only_sp else zone_shp

    tooltip_columns = ['NomeZona', 'mean_potential']
    tooltip_aliases = ['Zone', 'Mean Potential (' + reference + ')']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, 'Mean Potential (' + reference + ') ' + ' (' + variable + ')', 
                                 'YlOrBr', lambda x : x['mean_potential'], 
                          zones, tooltip_columns, tooltip_aliases)
    display(fmap)

In [11]:
# total of cyclable trips 
# considering the potential as the percentual of trips to be converted
# (0.3 potential, FE_VIA = 100 -> 30 trips would migrate to bicycle)

plot_potential_prob_widgets()

interactive(children=(Dropdown(description='Potential Variable', layout=Layout(width='500px'), options=(('Fina…

In [12]:
# jeito automático de calcular as interseções, faz o kernel morrer :(

#lines_per_polygon = gpd.overlay(od_trips, zone_shp, how = 'intersection')
#display(lines_per_polygon)

#lines_per_polygon = lines_per_polygon[['NumeroZona', 'FE_VIA', 'final_potential']]
#lines_per_polygon = lines_per_polygon.groupby(['NumeroZona'], as_index=False).sum()
#return geodf_polygons.merge(right = lines_per_polygon, on = polygon_id_column,
#                               how = 'left', suffixes = ("", "")).fillna(0)